# 7章

In [1]:
from dotenv import load_dotenv
from pathlib import Path

env_path = Path(".") / ".env"
load_dotenv(dotenv_path=env_path)  # 環境変数を読み込む

True

## 7.4, 7.5

バージョン違いで動かず

## 7.6

In [3]:
from langchain_community.document_loaders import GitLoader


def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/docs",
    repo_path="./docs",
    branch="main",
    file_filter=file_filter,
)

documents = loader.load()
print("Loaded documents:", len(documents))

Loaded documents: 1979


In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
import random
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
import os

# ----------------------
# 1. Embeddings と DB 初期化
# ----------------------
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
persist_dir = "./chroma_db"

# ----------------------
# 2. 既存DBがあるか確認
# ----------------------
if os.path.exists(persist_dir) and os.listdir(persist_dir):
    print("✅ 既存のChroma DBを読み込みます。")
    db = Chroma(embedding_function=embeddings, persist_directory=persist_dir)
else:
    print("⚙️ 新規にChroma DBを作成します。")

    # ----------------------
    # 2. サンプリング
    # ----------------------
    sample_fraction = 0.25
    sampled_documents = random.sample(
        documents, k=int(len(documents) * sample_fraction)
    )
    print("Sampled documents:", len(sampled_documents))

    # ----------------------
    # 3. チャンク化
    # ----------------------
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=4000,
        chunk_overlap=100,
    )
    split_docs = text_splitter.split_documents(sampled_documents)
    print("Split into chunks:", len(split_docs))

    # ----------------------
    # 4. Embeddings 作成 & Chroma登録
    # ----------------------
    db = Chroma(embedding_function=embeddings, persist_directory=persist_dir)

    from tqdm.notebook import tqdm

    batch_size = 100
    for i in tqdm(range(0, len(split_docs), batch_size)):
        batch = split_docs[i : i + batch_size]
        db.add_documents(batch)

    db.persist()
    print("✅ 新しいDBを保存しました。")


✅ 既存のChroma DBを読み込みます。


In [7]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈: """
{context}
"""

質問: {question}
''')

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

retriever = db.as_retriever()

chain = RunnableParallel(
    {
        "question": RunnablePassthrough(),
        "context": retriever,
    }
).assign(answer=prompt | model | StrOutputParser())


### フィードバックボタン

In [8]:
from uuid import UUID

import ipywidgets as widgets
from IPython.display import display
from langsmith import Client


def display_feedback_buttons(run_id: UUID) -> None:
    # GoodボタンとBadボタンを準備
    good_button = widgets.Button(
        description="Good",
        button_style="success",
        icon="thumbs-up",
    )
    bad_button = widgets.Button(
        description="Bad",
        button_style="danger",
        icon="thumbs-down",
    )

    # クリックされた際に実行される関数を定義
    def on_button_clicked(button: widgets.Button) -> None:
        if button == good_button:
            score = 1
        elif button == bad_button:
            score = 0
        else:
            raise ValueError(f"Unknown button: {button}")

        client = Client()
        client.create_feedback(run_id=run_id, key="thumbs", score=score)
        print("フィードバックを送信しました")

    # ボタンがクリックされた際にon_button_clicked関数を実行
    good_button.on_click(on_button_clicked)
    bad_button.on_click(on_button_clicked)

    # ボタンを表示
    display(good_button, bad_button)


In [9]:
from langchain_core.tracers.context import collect_runs

# LangSmithのトレースのID(Run ID)を取得するため、collect_runs関数を使用
with collect_runs() as runs_cb:
    output = chain.invoke("LangChainの概要を教えて")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)


LangChainは、主にJavaScript向けのSDKであり、さまざまな機能を提供するフレームワークです。公式のリファレンスは[こちら](https://reference.langchain.com/javascript/modules/langchain.html)で確認できます。また、LangChainに関する教育リソースやトレーニングは、[LangChain Academy](https://academy.langchain.com/)で提供されています。このアカデミーでは、LangChainの使い方や関連技術について学ぶことができます。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())

In [10]:
from langchain_core.tracers.context import collect_runs

# LangSmithのトレースのID(Run ID)を取得するため、collect_runs関数を使用
with collect_runs() as runs_cb:
    output = chain.invoke("CoTって何？")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)

文脈には「CoT」という用語に関する情報が含まれていないため、具体的な説明はできません。ただし、一般的に「CoT」は「Chain of Thought」の略で、思考の過程や推論の連鎖を指すことがあります。もし特定の文脈や詳細が必要であれば、追加の情報を提供してください。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())

In [11]:
from langchain_core.tracers.context import collect_runs

# LangSmithのトレースのID(Run ID)を取得するため、collect_runs関数を使用
with collect_runs() as runs_cb:
    output = chain.invoke("LangSmithって無料？")
    print(output["answer"])
    run_id = runs_cb.traced_runs[0].id

display_feedback_buttons(run_id)


LangSmithには無料のセルフホスティング版がありますが、基本機能にアクセスできるのみです。クラウドデプロイメントオプションやセルフホスティングのオプションは有料サービスです。詳細については、[LangSmithの価格ページ](https://www.langchain.com/pricing)を参照してください。


Button(button_style='success', description='Good', icon='thumbs-up', style=ButtonStyle())

Button(button_style='danger', description='Bad', icon='thumbs-down', style=ButtonStyle())